In [0]:
import pandas as pd
from py2neo import Graph, Node, Relationship

graph = Graph("bolt://localhost:7687",username='neo4j',password='yu15071253030')
data_path = "/Users/renpengju/Desktop/MEB/project_KB/project_info.xlsx"
data = pd.read_excel(data_path)

# node_1 = Node("Person",name="node_1")
# node_2 = Node("Person",name="node_2")
# graph.create(node_1)
# graph.create(node_2)

entities = ["shortdescription","applypeople","price",
            "Features","SafeLevel","ComplexLevel","operationstandard","taboocrowd",
            "operationadvantage","operationdefect","recoveries","painperception","preparation"]

name_dict = {"shortdescription":"项目简介","applypeople":"项目适应人群",
             "price":"项目价格","Features":"项目特点","SafeLevel":"手术安全度",
             "ComplexLevel":"手术复杂度","operationstandard":"功效","taboocrowd":"不适用人群",
             "operationadvantage":"手术优点","operationdefect":"手术缺点","recoveries":"恢复时间",
             "painperception":"手术疼痛感","preparation":"术前准备"}

for index, row in data.iterrows():
    # print(row["name"], row["shortdescription"])
    # print("\n")
    node_name = Node("项目名称", name=row["name"])  # 节点的label是"项目名称"，有name属性，属性值为具体的项目名
    graph.create(node_name)
    for entity in entities:

        node = Node(name_dict[entity],content=row[entity])
        node_rel = Relationship(node_name,name_dict[entity],node)
        graph.create(node)
        graph.create(node_rel)


In [39]:
import pandas as pd
from py2neo import Graph, Node, Relationship

graph = Graph("bolt://localhost:7687",username = "neo4j",password = "yu15071253030")
data_path = "./knowledge_base_hospital_data.xlsx"
data = pd.read_excel(data_path)

entities = ["hospitallevel","pricelevel","hospitalkind","abbreviation","hospitalscale","doctorstyle",
          "country","area","province","city_level","city_type"]

for index,row in data.iterrows():
    node_hospitalname = Node("hospitalname",name = row["hosptialname"])
    graph.create(node_hospitalname)
    for entity in entities:
        other_node = Node(entity,name = row[entity])
        node_relation =Relationship(node_hospitalname,entity,other_node)
        graph.create(other_node)
        graph.create(node_relation)

#建立同城关系
data_grouped = list(data.groupby("city"))
data = data_grouped[0][1]
selector = NodeMatcher(graph)
for group_id in range(len(data_grouped)):
    data = data_grouped[group_id][1]
    hospitalname_list = list(data["hosptialname"])
    for hospitalname in hospitalname_list:
        for hospitalname_same_city in hospitalname_list:
            if hospitalname_same_city != hospitalname:
                rel = Relationship(selector.match('hosptialname',name = hospitalname),'同城',selector.match('hosptialname', name = hospitalname_same_city))
                graph.create(rel)

In [19]:
import pandas as pd
from py2neo import Graph, Node, Relationship

graph = Graph("bolt://localhost:7687",username = "neo4j",password = "yu15071253030")
data_path = "./knowledge_base_data_20201027.xlsx"
data = pd.read_excel(data_path)

ziduan_str = "Name	Academic	AdaptiveSymptom	AestheticStandard	Alias	ApplyPeople	Description	DurationTime	Durations	Level	Methods	Narcosis	Notice	OperationAdvantage	OperationCourse	OperationDefect	OperationRisk	OperationStandard	Population	Preparation	Price	Recoveries	RecoveryTips	Shaping	ShortDescription	Stitches	SurgicalComplications	SurgicalMaterials	SurgicalSite	SwellingTime	TabooCrowd	Technicalmean	Therapeutics	TreatmentWay	Painperception	TypeAndCrowd"
entities_list = ziduan_str.strip().split("\t")

for index,row in data.iterrows():
    Name_node = Node("Name",content = row["Name"])
    graph.create(Name_node)
    for entity in entities_list:
        other_node = Node(entity,content = row[entity])
        node_relation = Relationship(Name_node,entity,other_node)
        graph.create(other_node)
        graph.create(node_relation)

In [18]:
len(data)

820

In [ ]:
import pandas as pd
from py2neo import Graph, Node, Relationship
graph = Graph("bolt://localhost:7687",username = "neo4j",password = "yu15071253030")
data_path = "./knowledge_base_hospital_data.xlsx"
data = pd.read_excel(data_path)
data_grouped = list(data.groupby("city"))

entities = ["hospitallevel","pricelevel","hospitalkind","abbreviation","hospitalscale","doctorstyle",
          "country","area","province","city_level","city_type"]
node_list = []
for group_id in range(len(data_grouped)):
    data = data_grouped[group_id][1]
    hospitalname_list = list(data["hosptialname"])
    for hospitalname in hospitalname_list:
        hospitalname_node = Node("hosptialname",content = hospitalname)
        graph.create(hospitalname_node)
        
        #建立同城关系
        for hospitalname_same_city in hospitalname_list:
            if hospitalname_same_city != hospitalname:
                hospitalname_same_city_node = Node("hosptialname",content = hospitalname_same_city)
                same_city_relation = Relationship(hospitalname_node,"同城",hospitalname_same_city_node)
                graph.create(hospitalname_same_city_node)
                graph.create(same_city_relation)
        
        #与各个字段建立关系
        col_data = data[data['hosptialname'] == hospitalname]
        for entity in entities:
            other_node = Node(entity,name = list(col_data[entity]))
            node_relation = Relationship(hospitalname_node,entity,other_node)
            graph.create(other_node)
            graph.create(node_relation)
            node_list.append(other_node)

In [8]:
import pandas as pd
from py2neo import Graph, Node, Relationship
graph = Graph("bolt://localhost:7687",username = "neo4j",password = "yu15071253030")
data_path = "./knowledge_base_hospital_data.xlsx"
data = pd.read_excel(data_path)

#获取医院名称列表
nodekey_list = []
for i in range(len(data)):
    nodekey_list.append(data["hosptialname"][i])
nodekey_list = list(set(nodekey_list))
# 获取医院属性列表
nodevalue_list = []
for i in range(len(data)):
    for j in range(1,len(data.columns)):
        nodevalue_list.append(data[data.columns[j]][i])
nodevalue_list = list(set(nodevalue_list))
nodevalue_list = [str(item) for item in nodevalue_list]

#获取实体1，实体2，关系数据
links_dict = {}
name_list = []
relation_list = []
name2_list = []
for i in range(0, len(data)):
    m = 0
    name_node = data[data.columns[m]][i]
    while m < len(data.columns)-1:
        relation_list.append(data.columns[m+1])
        name2_list.append(data[data.columns[m+1]][i])
        name_list.append(name_node)
        m += 1
# 将数据中int类型全部转成string
name_list = [str(i) for i in name_list]
name2_list = [str(i) for i in name2_list]
# 整合数据，将三个list整合成一个dict
links_dict['name'] = name_list
links_dict['relation'] = relation_list
links_dict['name2'] = name2_list
# 将数据转成DataFrame
df_data = pd.DataFrame(links_dict)

# 创建节点
for name in nodekey_list:
    name_node = Node('医院名称', name=name)
    graph.create(name_node)
for name in nodevalue_list:
    value_node = Node('属性值', name=name)
    graph.create(value_node)
#创建关系
selector = NodeMatcher(graph)
for m in range(0, len(df_data)):
    rel = Relationship(selector.match('医院名称',name = df_data['name'][m]),
                        df_data['relation'][m],
                        selector.match('属性值', name = df_data['name2'][m]))
    graph.create(rel)

#获取同城关系csv数据
sub_node = []
rel_node = []
obj_node = []
data_grouped = list(data.groupby("city"))
for group_id in range(len(data_grouped)):
    data = data_grouped[group_id][1]
    hospitalname_list = list(data["hosptialname"])
    for hospitalname in hospitalname_list:
        for hospitalname_same_city in hospitalname_list:
            if hospitalname_same_city != hospitalname:
                sub_node.append(hospitalname)
                rel_node.append('同城')
                obj_node.append(hospitalname_same_city)
data = {'subject':sub_node,'relation':rel_node,'object':obj_node}
pd.DataFrame(data).to_csv('./rel_data.csv')

#获取医院名称、id csv数据
name_data = list(data['hosptialname'])
id = []
for i in range(len(name_data)):
    id.append(i)
name_data = {'hospitalname':name_data,'id':id}
pd.DataFrame(name_data).to_csv("./node_data.csv")